In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd

In [2]:
# review link link
link = 'https://play.google.com/store/apps/details?id=com.kbstar.kbland&showAllReviews=true'

In [3]:
# how many scrolls we need
scroll_cnt = 4

In [4]:
driver = webdriver.Chrome("C:/chrome/chromedriver.exe")
driver.get(link)

In [5]:
os.makedirs('result', exist_ok=True)

In [6]:
while True:
    # (1) 4번의  스크롤링
    for i in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)

    # (2) 더보기 클릭
    try:
        driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    except:
        print('끝')
        break

Cannot find load more button...


In [7]:
# get review containers
reviews = driver.find_elements_by_xpath('//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]')

In [8]:
print('There are %d reviews avaliable!' % len(reviews))
print('Writing the data...')

There are 262 reviews avaliable!
Writing the data...


In [9]:
# create empty dataframe to store data
df = pd.DataFrame(columns=['date', 'ratings', 'helpful', 'comment'])

In [10]:
# get review data
for review in reviews:
    # parse string to html using bs4
    soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')

    # review date
    date = soup.find(class_='p2TkOb').text
    date = datetime.strptime(date, '%Y년 %m월 %d일')
    date = date.strftime('%Y-%m-%d')

    # ratiabsng
    ratings = int(soup.find('div', role='img').get('aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip())

    # helpful
    helpful = soup.find(class_='jUL89d y92BAb').text
    if not helpful:
        helpful = 0
  
    # review text
    comment = soup.find('span', jsname='fbQN7e').text
    if not comment:
        comment = soup.find('span', jsname='bN97Pc').text

    # developer comment
    developer_comment = None
    dc_div = soup.find('div', class_='LVQB0b')
    if dc_div:
        developer_comment = dc_div.text.replace('\n', ' ')

    # append to dataframe
    df = df.append({
        'date': date,
#         'name': name,
        'ratings': ratings,
        'helpful': helpful,
        'comment': comment,
#         'developer_comment': developer_comment
    }, ignore_index=True)

In [11]:
# finally save the dataframe into csv file
df.to_csv('리브온_Playstore.csv', encoding='utf-8-sig', index=False)
driver.stop_client()
driver.close()

print('Done!')

Done!


In [12]:
train = pd.read_csv('리브온_Playstore.csv')
train

,date,ratings,helpful,comment
0,2020-11-08,5,4,"매매,전세,월세 가격이 표시되면 훨씬더 좋을것같습니다~~"
1,2020-09-28,1,32,앱 UX 최악. 시세조회때문에 어쩔 수 없이 쓰지만 쓸 일 없어지면 바로 삭제 하고...
2,2020-08-26,1,33,"전혀 개선이 안되는데, 사용도 안해보는건가? 최소한 조회결과에 대해 추천순이든 낮은..."
3,2020-10-12,1,2,응답코드 오류만 나와요 ㅠ
4,2020-08-30,1,14,와... 너무 느려요. 게다가 조회도 gui 스레드에서 해서 화면에 anr 걸리고 ...
...,...,...,...,...
257,2020-04-24,5,0,좋아용
258,2018-02-19,5,7,좋아요
259,2019-02-02,5,1,좋네요
260,2018-06-16,5,2,굿
